<a href="https://colab.research.google.com/github/ShykerBogdan/Prozzoro_project/blob/master/Data_prepr_batches_100_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import time
from datetime import datetime
import numpy as np

In [4]:
filename='/content/gdrive/My Drive/ML_proz/data_source_cleaned.csv'
df=pd.read_csv(filename)
df.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,5,14,15,16,17,18,24,38,48,50,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3507778, 57)

In [5]:
cpv=pd.read_csv('/content/gdrive/My Drive/ML_proz/cpv (2).csv')
cpv.shape

(2687175, 3)

In [0]:
df=pd.merge(df,cpv,left_on='tenders.id',right_on='tender_id',how='left')

In [0]:
df['common_name']=df.apply(lambda raw: raw['tenderers_name'] if pd.notnull(raw['tenderers_name']) else (raw['awards.suppliers_identifier_legalName'] if raw['awards.suppliers_identifier_legalName']!='\\N' else raw['awards.suppliers_name']),axis=1)

In [146]:
df.columns

Index(['awards.bid_id', 'awards.count_documents', 'awards.date', 'awards.id',
       'awards.status', 'awards.suppliers_identifier_id',
       'awards.suppliers_identifier_legalName', 'awards.suppliers_name',
       'awards.value_amount', 'contracts.awardID', 'contracts.date',
       'contracts.dateSigned', 'contracts.id', 'contracts.status',
       'contracts.value_amount', 'lots.auctionPeriod_startDate',
       'lots.guarantee_amount', 'lots.guarantee_currency', 'lots.id',
       'lots.status', 'lots.value_amount', 'lots.value_valueAddedTaxIncluded',
       'tender_id_x', 'tenderers_address_postalCode',
       'tenderers_identifier_id', 'tenderers_name',
       'tenders.auctionPeriod_startDate', 'tenders.awardCriteria',
       'tenders.awardPeriod_startDate', 'tenders.complaintPeriod_endDate',
       'tenders.complaintPeriod_startDate', 'tenders.enquiryPeriod_endDate',
       'tenders.enquiryPeriod_startDate', 'tenders.guarantee_amount',
       'tenders.id', 'tenders.owner', 'tenders

In [0]:
class DataPreproccessing:
    def __init__(self, data_source, churn_interval=90, start_period=None, end_period=None,start_data_unique=None):
       
        data_source=data_source[pd.to_datetime(data_source['new_date'])>=start_period]
        data_source=data_source[pd.to_datetime(data_source['new_date'])<end_period]
        self.start_period=start_period
        self.end_period=end_period
        self.start_data_unique=start_data_unique
        self.last_day = pd.to_datetime('2018-11-29')
        self.divide_data = pd.to_datetime(self.end_period) - pd.DateOffset(days=churn_interval)
        print(self.divide_data)
        self.data_source = data_source[pd.to_datetime(data_source['new_date']) < self.divide_data]
        
        self.label_data = data_source[pd.to_datetime(data_source['new_date']) >= self.divide_data]
        # вибираємо унікальних постачальників
#         self.unique_id_data=self.data_source[(pd.to_datetime(self.data_source['new_date'])>=self.start_data_unique) & (pd.to_datetime(self.data_source['new_date'])<=self.divide_data)]
        self.unique_data=self.data_source[pd.to_datetime(self.data_source['new_date'])>=self.start_data_unique]
        self.unique_data=self.unique_data[pd.to_datetime(self.unique_data['new_date'])<self.divide_data]
        self.unique_id = list(self.unique_data['participants'].value_counts().index)
        # створюємо dataframe з колонкою унікальних постачальників і туди будемо додавати features
        self.feature_data = pd.DataFrame(self.unique_id, columns=['unique_id'])
        
    def print_dates(self):
        print('{0}:'.format((pd.to_datetime(self.divide_data)-pd.to_datetime(self.start_data_unique)).days))
        print('{0}:'.format((pd.to_datetime(self.end_period)-pd.to_datetime(self.start_data_unique)).days))
        print('Features: [{0}/{1})'.format(self.start_period,self.divide_data))
        print('Unique_id: [{0}/{1})'.format(self.start_data_unique,self.divide_data))
        print('Labeling: [{0}/{1}]'.format(self.divide_data,self.end_period))
    
    
    
    def cleaning(self):
        
        self.data_source['tenders.procuringEntity_address_postalCode']=self.data_source['tenders.procuringEntity_address_postalCode'].astype(str).str[:2]
        self.data_source['tenderers_address_postalCode']=self.data_source['tenderers_address_postalCode'].astype(str).str[:2]
        table_postal_code={'Київ':['01', '02', '03', '04', '05', '06'],
                  'Київська область':['07', '08', '09'],
                  'Житомирська область':['10', '11', '12', '13'],
                  'Черніговська область':['14', '15', '16', '17'],
                  'Черкаська область':['18', '19', '20'],
                  'Вінницька область':['21', '22', '23', '24'],
                  'Кіровоградська область':['25', '26', '27', '28'],
                  'Хмельницька область':['29', '30', '31', '32'],
                  'Рівненська область': ['33', '34', '35'],
                  'Полтавська область':['36', '37', '38', '39'],
                  'Сумська область':['40', '41', '42'],
                  'Волинська область':['43', '44', '45'],
                  'Тернопольська область':['46', '47', '48'],
                  'Дніпропетровська область':['49', '50', '51', '52', '53'],
                  'Миколаївська область':['54', '55', '56', '57'],
                  'Чернівецька область':['58', '59', '60'],
                  'Харьківська область':['61', '62', '63', '64'],
                  'Одеська область':['65', '66', '67', '68'],
                  'Запоріжська область':['69', '70', '71', '72'],
                  'Херсонська область':['73', '74', '75'],
                  'Івано-Франківська область':['76', '77', '78'],
                  'Львівська область':['79', '80', '81', '82'],
                  'Донецька область':['83', '84', '85', '86', '87'],
                  'Закарпатська область':['88', '89', '90'],
                  'Луганська область':['91', '92', '93', '94'],
                  'Автономна республіка Крим':['95', '96', '97', '98'],
                  'Севастопіль':['99']}
        regions=dict()
        for key,value in table_postal_code.items():
            for item in value:
              regions[item]=key
        def get_region(raw,regions):
            try:
              raw['region']=regions[raw['tenders.procuringEntity_address_postalCode']]
            except:  
              raw['region']=np.nan
            return raw
        self.data_source=self.data_source.apply(get_region,args=(regions,),axis=1)
        def get_region_supp(raw,regions):
            try:
              raw['region_supp']=regions[raw['tenderers_address_postalCode']]
            except:  
              raw['region_supp']=np.nan
            return raw
        self.data_source=self.data_source.apply(get_region_supp,args=(regions,),axis=1)
        print('regions updated')
        
    def split_into_batches(self, n=20000, filename=None):
        
        self.cleaning()
        feature_batches = list()
        n_iter = int(np.ceil(self.feature_data.shape[0] / n))
        for i in range(n_iter):
            start_time = time.time()
            print('Batch {0}-{1}:'.format(i * n, (i + 1) * n))
            unique_id = self.feature_data.iloc[i * n:(i + 1) * n]['unique_id'].values
            feature_df = pd.DataFrame(unique_id, columns=['unique_id'])
            feature_batches.append(feature_df)
            # part of data_source
            df_ = self.data_source.loc[self.data_source['participants'].isin(unique_id)]
            print('Unique_supp:{0}-{1}'.format(len(unique_id), df_.shape[0]))
            df_ = df_.sort_values(by='new_date')
            self.divide_data = str(self.divide_data)[:10]

            # features:
#             def get_most_common_name(raw,df):
#                 data=df[df['participants']==raw['unique_id']][['awards.suppliers_identifier_legalName','tenderers_name','awards.suppliers_name']]
#                 data['common_name']=data.apply(lambda raw: raw['tenderers_name'] if pd.notnull(raw['tenderers_name']) else (raw['awards.suppliers_identifier_legalName'] if raw['awards.suppliers_identifier_legalName']!='\\N' else raw['awards.suppliers_name']),axis=1)
#                 return raw
#             feature_batches[i] = feature_batches[i].apply(get_most_common_name, args=(df_[['participants','awards.suppliers_identifier_legalName','tenderers_name','awards.suppliers_name']],),axis=1)
#             # 0. First activity
            # 1. Last activity
            def get_last_activity(raw, data,eps=0.7):
                date=data[data['participants'] == raw['unique_id']]
                dates=date['new_date'].values
                # favourite host
                try:
                  if date['tenders.procuringEntity_identifier_id'].value_counts().iloc[0]/date.shape[0] >=eps:
                    raw['favourite_host']=1
                  else:
                    raw['favourite_host']=0
                  raw['count_unique_hosts']=date['tenders.procuringEntity_identifier_id'].value_counts().shape[0]
                except:
                  raw['favourite_host']=np.nan
                  raw['count_unique_hosts']=np.nan
                # name
                raw['common_name']=list(date['common_name'].value_counts().index)[0]
                # get cpv
                raw['cpv']=','.join([str(item) for item in set(date['cpv2'].values)])
                # unique_region
                raw['unique_regions']=len(set(date['region'].values))
                # get supp region
                try:
                  raw['supp_region']=list(date['region_supp'].value_counts().index)[0]
                except:
                  raw['supp_region']=np.nan
#                 raw['last_activity_date'] = dates[-1]
                try:
                    raw['first_activity_days'] = (
                        pd.to_datetime(self.divide_data) - pd.to_datetime(dates[0])).days
                    raw['last_activity_days'] = (
                        pd.to_datetime(self.divide_data) - pd.to_datetime(dates[-1])).days
                    if len(dates)!=1:
                      raw['average_activity']=(pd.to_datetime(dates[-1])-pd.to_datetime(dates[0])).days/(len(dates)-1)
                    else:
                      raw['average_activity']='?'
                except:
                    print('--------------')
                    raw['average_activity'] = '?'
                    
                return raw

            feature_batches[i] = feature_batches[i].apply(get_last_activity, args=(df_[['common_name','cpv2','participants','new_date','tenders.procuringEntity_identifier_id','tenders.id','region','region_supp']],0.75,),
                                                          axis=1)
            feature_batches[i]['last_activity_days'] = feature_batches[i].apply(
                lambda x: 0 if x['last_activity_days'] < 0 else x['last_activity_days'], axis=1)
            print(" --- %s seconds ---" % (time.time() - start_time))

            # 2. Average activity
#             def get_average_activity(raw, data):
#                 dates_ = list(data[data['participants'] == raw['unique_id']]['new_date'].values)
#                 try:
#                     raw['average_activity'] = (pd.to_datetime(dates_[-1]) - pd.to_datetime(dates_[0])).days / len(dates_)
#                 except:
#                     raw['average_activity'] = np.nan
#                 return raw

#             feature_batches[i] = feature_batches[i].apply(get_average_activity, args=(df_[['participants','new_date']],),
#                                                           axis=1)

            # 3. Count lots
            freq = df_['participants'].value_counts()

            def count_lots(raw, frequency):
                raw['count_lots'] = frequency[raw['unique_id']]
                return raw

            feature_batches[i] = feature_batches[i].apply(count_lots, args=(freq,), axis=1)

            # 4. Win, lose
            frequency = df_[df_['winner'] == 1]['participants'].value_counts()
            print(" --- %s seconds ---" % (time.time() - start_time))
            def win_lose(raw, frequency):
                # use try bsc if index not in frequency 'win'=0
                try:
                    raw['win'] = frequency[raw['unique_id']]
                except:
                    raw['win'] = 0
                return raw

            feature_batches[i] = feature_batches[i].apply(win_lose, args=(
                frequency,), axis=1)
            feature_batches[i]['lose'] = feature_batches[i]['count_lots'] - feature_batches[i]['win']
            print(" --- %s seconds ---" % (time.time() - start_time))
#             5. Count win open

            frequency = df_.loc[(df_['winner'] == 1) & (df_['tenders.procurementMethod'] == 'open')][
                'participants'].value_counts()

            def count_win_open(raw, frequency):
                try:
                    raw['win_open'] = frequency[raw['unique_id']]
                except:
                    raw['win_open'] = 0
                return raw

            feature_batches[i] = feature_batches[i].apply(count_win_open, args=(
                frequency,), axis=1)
            feature_batches[i]['win_not_open'] = feature_batches[i]['win'] - feature_batches[i]['win_open']
            print(" --- %s seconds ---" % (time.time() - start_time))
#             6. Count lose open
            frequency = df_.loc[(df_['winner'] == 0) & (df_['tenders.procurementMethod'] == 'open')][
                'participants'].value_counts()

            def count_lose_open(raw, frequency):
                try:
                    raw['lose_open'] = frequency[raw['unique_id']]
                except:
                    raw['lose_open'] = 0
                return raw

            feature_batches[i] = feature_batches[i].apply(count_lose_open, args=(
                frequency,), axis=1)
#             feature_batches[i]['lose_not_open'] = feature_batches[i]['lose'] - feature_batches[i]['lose_open']
            print(" --- %s seconds ---" % (time.time() - start_time))
            # 7. Average economy
            data = df_[df_['winner'] == 1][
                ['participants', 'awards.value_amount', 'lots.value_amount']]
            data['economy_value'] = data['lots.value_amount'] - data['awards.value_amount']
            data['economy_percent'] = data['awards.value_amount'] / data['lots.value_amount']

            def get_economy(raw, data):
                d = data[data['participants'] == raw['unique_id']]
                raw['economy_value'] = np.nanmean(d['economy_value'])
                raw['average_winner_price']=np.nansum(d['awards.value_amount'])/d.shape[0]
                raw['economy_percent'] = np.nanmean(d['economy_percent'])
                return raw

            feature_batches[i] = feature_batches[i].apply(get_economy, args=(data,), axis=1)
#             8. Get label
            participants = list(self.label_data['participants'].value_counts().index)

            def get_y(raw, participants):
              if raw['unique_id'] in participants:
                  raw['y'] = 1
              else:
                  raw['y'] = 0
              return raw
#             9. Get favourite hosts
            feature_batches[i] = feature_batches[i].apply(get_y, args=(participants,), axis=1)
#             def favourite_percent(raw, data, eps = 0.7):
#                 d = data[data['participants']==raw['unique_id']].drop_duplicates('tenders.id')
#                 try:
#                   if d['tenders.procuringEntity_identifier_id'].value_counts().iloc[0]/d.shape[0] >=eps:
#                     raw['favourite_host']=1
#                   else:
#                     raw['favourite_host']=0
#                     raw['count_unique_hosts']=d['tenders.procuringEntity_identifier_id'].value_counts().shape[0]
#                 except:
#                   raw['favourite_host']=np.nan
#                   raw['count_unique_hosts']=np.nan
#                 del d
#                 return raw
#             feature_batches[i] = feature_batches[i].apply(favourite_percent, args=(df_[['participants', 'tenders.id', 'tenders.procuringEntity_identifier_id']],0.75,), axis=1)
            # 10. Get unique region
            print(" Batch %s seconds ---" % (time.time() - start_time))
        if filename:
            print(f'write to csv')
            feature_batches[i].to_csv(filename + '{0}'.format(i) + '.csv', index=False)

        
        # concat dataframes into one
        print('Concat')
        all_batches_df = pd.concat(feature_batches)
        self.feature_data = all_batches_df.copy()
        if filename:
            print('write to csv_all')
            all_batches_df.to_csv(filename + '_all.csv', index=False)
        print(" --- %s seconds ---" % (time.time() - start_time))
    def get_label(self):
        participants = list(self.label_data['participants'].value_counts().index)

        def get_y(raw, participants):
            if raw['unique_id'] in participants:
                raw['y'] = 1
            else:
                raw['y'] = 0
            return raw

        self.feature_data = self.feature_data.apply(get_y, args=(participants,), axis=1)


In [0]:
col=['winner','tenders.auctionPeriod_startDate','contracts.dateSigned',
     'awards.date','participants','date_from_id','contracts.value_amount','tenders.procurementMethod','contracts.status','awards.value_amount','tenders.id','tenders.procuringEntity_identifier_id','lots.value_amount']

In [0]:
start_period = '2016-09-10'
start_data_unique = ['2016-09-10', '2016-12-09', '2017-03-09', '2017-06-07', '2017-09-05', '2017-12-04']
end_period = ['2017-03-09', '2017-06-07', '2017-09-05', '2017-12-04', '2018-03-04', '2018-06-02']

In [18]:
for i in range(len(end_period)):
  prepr = DataPreproccessing(df.iloc[:100000],start_period='2016-09-10',end_period=end_period[i],start_data_unique=start_data_unique[i],churn_interval=90)
  prepr.print_dates()
  prepr.split_into_batches(n=10000)
  if i>0:
    print('i={0}'.format(i))
    print('Bad rows : {0}'.format(prepr.feature_data[prepr.feature_data['unique_id'].isin(z)].shape[0]))
  z=prepr.feature_data[prepr.feature_data['y']==0]['unique_id'].values

2016-12-09 00:00:00
90:
180:
Features: [2016-09-10/2016-12-09 00:00:00)
Unique_id: [2016-09-10/2016-12-09 00:00:00)
Labeling: [2016-12-09 00:00:00/2017-03-09]
regions updated
Batch 0-10000:
Unique_supp:4748-6723
 --- 46.85681891441345 seconds ---
 --- 50.36337423324585 seconds ---
 --- 53.67832612991333 seconds ---
 --- 57.270970582962036 seconds ---
 --- 60.810346364974976 seconds ---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:229: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:230: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:231: RuntimeWarning: Mean of empty slice


 Batch 82.22820830345154 seconds ---
Concat
 --- 82.23245167732239 seconds ---
2017-03-09 00:00:00
90:
180:
Features: [2016-09-10/2017-03-09 00:00:00)
Unique_id: [2016-12-09/2017-03-09 00:00:00)
Labeling: [2017-03-09 00:00:00/2017-06-07]
regions updated
Batch 0-10000:
Unique_supp:5742-11117
 --- 61.13369369506836 seconds ---
 --- 65.36326098442078 seconds ---
 --- 69.31244325637817 seconds ---
 --- 73.60533356666565 seconds ---
 --- 77.83743214607239 seconds ---
 Batch 107.11109972000122 seconds ---
Concat
 --- 107.11607336997986 seconds ---
i=1
Bad rows : 0
2017-06-07 00:00:00
90:
180:
Features: [2016-09-10/2017-06-07 00:00:00)
Unique_id: [2017-03-09/2017-06-07 00:00:00)
Labeling: [2017-06-07 00:00:00/2017-09-05]
regions updated
Batch 0-10000:
Unique_supp:6567-15235
 --- 76.11322021484375 seconds ---
 --- 80.93637490272522 seconds ---
 --- 85.68679857254028 seconds ---
 --- 90.30061531066895 seconds ---
 --- 95.43443036079407 seconds ---
 Batch 132.8853349685669 seconds ---
Concat
 --

In [21]:
prepr.feature_data.head()

,average_activity,common_name,count_unique_hosts,cpv,favourite_host,first_activity_days,last_activity_days,supp_region,unique_id,unique_regions,count_lots,win,lose,win_open,win_not_open,lose_open,economy_value,average_winner_price,economy_percent,y
0,0.659794,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ЛІВАЙ...",95,9,0,65,1,NaN,41449359,24,98,54,44,14,40,44,900.045370,1.212868e+05,0.986368,1
1,3.70423,"ПАТ ""Укртелеком""",136,"32,64,70,72,60,92",0,529,3,Дніпропетровська область,21560766,26,143,139,4,3,136,4,1828.714532,2.185948e+05,0.981465,1
2,1.59819,"ТОВ ""Епіцентр К""",254,"32,33,34,35,37,39,9,42,43,44,45,14,15,16,18,19...",0,531,2,NaN,32490244,30,332,192,140,63,129,140,4611.493281,4.986688e+04,0.940641,1
3,2.38272,"ПП ""ОККО КОНТРАКТ""",152,9,0,388,2,Київ,36248687,27,163,56,107,37,19,107,36396.738036,4.595428e+05,0.936587,1
4,4,"ПАТ ""Київенерго""",111,"9,50",0,500,4,NaN,00131305,1,125,125,0,0,125,0,16586.793520,1.970628e+06,0.990776,1


In [28]:
df[df['participants']=='32490244'][['tenderers_address_postalCode']]

,tenderers_address_postalCode
39,NaN
210,NaN
262,02139
360,NaN
420,02139
554,02139
588,NaN
718,NaN
723,02139
835,02139
